<a href="https://colab.research.google.com/github/mahhhdy/thesis/blob/main/Classification_data_train_and_test_splite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


this post has been used </br>
https://github.com/NourozR/Stock-Price-Prediction-LSTM/blob/1829ad5c8b3ece7b0e9c16fd76cf2550142683b2/StockPricePrediction.py#L11

In [33]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense , Activation, Dropout
from keras.layers import LSTM , SimpleRNN
from keras.layers import BatchNormalization
from sklearn.metrics import mean_squared_error
from google.colab import drive
from google.colab import files
# from keras.layers.normalization import batch_normalization

from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

np.random.seed(7)

########## import dataset ##########
drive.mount('/content/drive')
path='/content/drive/MyDrive/DataSets/BTCPrediction/15XAUUSD.csv'
# dataset = pd.read_csv(path,usecols=[1,2,3,4])
dataset = pd.read_csv(path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
dataset = dataset.rename({'Time':'Date','|PD|':'PD'},axis=1)

In [35]:
dataset = dataset.reindex(index = dataset.index[::-1])

In [36]:
dataset = dataset.head(20)

In [37]:
obs = np.arange(1, len(dataset) + 1, 1)
obs

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])

In [38]:
dataset['Average'] = dataset[['Open','Close','Low','High']].mean(axis=1)
# dataset

In [39]:
dataset['State'] =( dataset['Close'] > dataset['Open']).astype(int)
dataset

,Date,Open,High,Low,Close,PD,PD%,ATR,PD% / ATR,Average,State
1999,4/13/2022 7:00,1969.85,1969.89,1968.79,1969.83,0.02,0.001015,1.678750,0.000605,1969.5900,0
1998,4/13/2022 7:15,1969.83,1970.20,1968.71,1969.82,0.01,0.000508,1.681250,0.000302,1969.6400,0
1997,4/13/2022 7:30,1969.82,1970.53,1969.43,1969.59,0.23,0.011676,1.685417,0.006928,1969.8425,0
1996,4/13/2022 7:45,1969.62,1969.63,1967.69,1968.26,1.36,0.069049,1.730833,0.039893,1968.8000,0
1995,4/13/2022 8:00,1968.26,1969.20,1967.60,1969.19,0.93,-0.047250,1.723333,-0.027418,1968.5625,1
1994,4/13/2022 8:15,1969.17,1969.73,1968.40,1969.40,0.23,-0.011680,1.760833,-0.006633,1969.1750,1
1993,4/13/2022 8:30,1969.39,1973.41,1969.30,1973.23,3.84,-0.194984,1.885000,-0.103440,1971.3325,1
1992,4/13/2022 8:45,1973.23,1973.50,1971.53,1972.07,1.16,0.058787,1.935000,0.030381,1972.5825,0
1991,4/13/2022 9:00,1972.01,1972.01,1969.42,1969.76,2.25,0.114097,1.908750,0.059776,1970.8000,0
1990,4/13/2022 9:15,1969.78,1971.10,1967.20,1967.63,2.15,0.109149,1.973750,0.055300,1968.9275,0


In [40]:
# TAKING DIFFERENT INDICATORS FOR PREDICTION
OHLC_avg = dataset[['Average']].to_numpy()
# HLC_avg = dataset[['High', 'Low', 'Close']].mean(axis = 1)
close_val = dataset[['State']].to_numpy()

In [54]:
def new_dataset(dataset, target , step_size):
	data_X, data_Y = [], []
	for i in range(len(dataset)-step_size):
		a = dataset[i:(i+step_size), 0]
		data_X.append(a)
		data_Y.append(target[i + step_size])
	return np.array(data_X), np.array(data_Y)

In [64]:
X, Y = new_dataset(OHLC_avg, close_val , 1)

In [65]:
X.shape , Y.shape

((19, 1), (19, 1))

In [66]:
X

array([[1969.59  ],
       [1969.64  ],
       [1969.8425],
       [1968.8   ],
       [1968.5625],
       [1969.175 ],
       [1971.3325],
       [1972.5825],
       [1970.8   ],
       [1968.9275],
       [1968.0475],
       [1968.9375],
       [1967.5625],
       [1966.6075],
       [1968.1   ],
       [1970.7725],
       [1971.5725],
       [1973.6875],
       [1974.585 ]])

In [67]:
Y

array([[0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0]])

In [68]:
len_train = int(len(X) * 0.8)
len_test = len(X) - len_train

trainX, testX = X[0:len_train,:], X[len_train:len(OHLC_avg),:]

trainY, testY = Y[0:len_train],Y[len_train:len(OHLC_avg)]

In [69]:
trainX , len(trainX)

(array([[1969.59  ],
        [1969.64  ],
        [1969.8425],
        [1968.8   ],
        [1968.5625],
        [1969.175 ],
        [1971.3325],
        [1972.5825],
        [1970.8   ],
        [1968.9275],
        [1968.0475],
        [1968.9375],
        [1967.5625],
        [1966.6075],
        [1968.1   ]]), 15)

In [70]:
trainY , len(trainY)

(array([[0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1]]), 15)

In [71]:
testX

array([[1970.7725],
       [1971.5725],
       [1973.6875],
       [1974.585 ]])

In [72]:
testY , len(trainY)

(array([[1],
        [1],
        [0],
        [0]]), 15)